<a href="https://colab.research.google.com/github/CodeHunterOfficial/CCPiOIIC/blob/main/ML/ArabovMK_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

Вам нужно написать следующий пайплайн:
1. Получить эмбединги пользователей с помощью SVD.
2. Подать их в нейронную сеть, которую вы делали на практическом занятии.
Она принимает на вход пользователей и эмбединг айтомов, далее объединяет их в два полносвязанных слоя. На последнем слое выдаёт прогноз с функцией активации сигмоид.
3. Обучить сеть.


In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

### Шаг 1: Получение эмбедингов пользователей с помощью SVD

# Пример матрицы взаимодействий (пользователи x айтемы)
# Например, это могут быть рейтинги пользователей для фильмов
user_item_matrix = np.random.randint(0, 6, size=(100, 50))  # 100 пользователей, 50 айтемов

# Преобразуем матрицу в тип данных с плавающей точкой
user_item_matrix = user_item_matrix.astype(np.float64)

# Применяем SVD для получения эмбедингов пользователей
k = 10  # Размерность эмбедингов
U, sigma, Vt = svds(user_item_matrix, k=k)

# Нормализуем сингулярные значения
sigma = np.diag(sigma)
user_embeddings = np.dot(U, sigma)

### Шаг 2: Создание нейронной сети

class NeuralNetwork(nn.Module):
    def __init__(self, user_embedding_dim, item_embedding_dim, hidden_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(user_embedding_dim + item_embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, user_embedding, item_embedding):
        x = torch.cat((user_embedding, item_embedding), dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Параметры модели
user_embedding_dim = k  # Размерность эмбедингов пользователей
item_embedding_dim = 10  # Размерность эмбедингов айтемов (можно использовать SVD для айтемов тоже)
hidden_dim = 64

# Создаем модель
model = NeuralNetwork(user_embedding_dim, item_embedding_dim, hidden_dim)

### Шаг 3: Подготовка данных для обучения

# Пример данных для обучения
num_samples = 1000  # Количество пар (пользователь, айтем)
user_indices = np.random.randint(0, 100, size=num_samples)  # Случайные пользователи
item_indices = np.random.randint(0, 50, size=num_samples)   # Случайные айтемы
targets = np.random.randint(0, 2, size=num_samples)         # Бинарные метки (0 или 1)

# Получаем эмбединги для выбранных пользователей и айтемов
user_embeddings_selected = user_embeddings[user_indices]
item_embeddings = np.random.randn(50, item_embedding_dim)  # Эмбединги айтемов
item_embeddings_selected = item_embeddings[item_indices]

# Преобразуем данные в тензоры PyTorch
user_embeddings_tensor = torch.tensor(user_embeddings_selected, dtype=torch.float32)
item_embeddings_tensor = torch.tensor(item_embeddings_selected, dtype=torch.float32)
targets_tensor = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

### Шаг 4: Обучение сети

# Функция потерь и оптимизатор
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Списки для хранения значений потерь и точности
losses = []
accuracies = []

# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Прогоняем данные через модель
    outputs = model(user_embeddings_tensor, item_embeddings_tensor)
    loss = criterion(outputs, targets_tensor)

    # Обратное распространение и оптимизация
    loss.backward()
    optimizer.step()

    # Вычисляем accuracy
    predictions = (outputs > 0.5).float()  # Преобразуем вероятности в бинарные предсказания
    correct = (predictions == targets_tensor).float().sum()
    accuracy = correct / targets_tensor.shape[0]

    # Сохраняем значения потерь и точности
    losses.append(loss.item())
    accuracies.append(accuracy.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}')

### Шаг 5: Визуализация процесса обучения

# График потерь
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(losses, label='Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# График точности
plt.subplot(1, 2, 2)
plt.plot(accuracies, label='Accuracy', color='orange')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()